In [1]:
# !pip -q install python-crfsuite
# !pip -q install category_encoders
# !pip -q install flair
# import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('words')
# # !pip3 install torch==1.3.1+cpu torchvision==0.4.2+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [28]:
import pycrfsuite
import nltk, re, math
from nltk import tag
import pandas as pd
import numpy as np
import collections, itertools
from flair.data import Sentence
from flair.models import SequenceTagger
flatten = itertools.chain.from_iterable
TAGGER = SequenceTagger.load('pos')
POS_DICTIONARY = {}

2019-12-11 06:33:31,802 loading file /home/nehas/.flair/models/en-pos-ontonotes-v0.4.pt


In [3]:
# the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
from sklearn.feature_extraction.text import CountVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [4]:
def createDiagActFeatures(sentence):
    features = []
    if len(sentence.split())>=2:
        tagged_sent = tag.pos_tag(nltk.word_tokenize(sentence))
        for tagset in tagged_sent:
            features.append('TOKEN_'+tagset[0])
        for tagset in tagged_sent:
            features.append('POS_'+tagset[1])
        for words in sentence.split():
            features.append(words)
        features.append('/')
    return [[features]]

In [5]:
def flairGetPOSTag(d_words):
    if d_words in POS_DICTIONARY:
        return POS_DICTIONARY[d_words]
    else:
        d = []
        preprocessed_text = d_words.replace(',', ', ').replace('.', '. ')
        sentence = Sentence(preprocessed_text, use_tokenizer=True)
        TAGGER.predict(sentence)
        for token in sentence:
            pos = token.get_tag('pos').value
            d.append(pos)
        POS_DICTIONARY[d_words] = d
        return d

In [6]:
tagger = pycrfsuite.Tagger()
tagger.open('model/model.crf.tagger')

In [7]:
# xTest = createDiagActFeatures("I can imagine.")
# yPred = [tagger.tag(xseq) for xseq in createDiagActFeatures("I can imagine.")][0][0]

In [8]:
df = pd.read_csv("train_test/training.csv",names=['text','character','gender'])

In [9]:
df.shape

(10113, 3)

In [10]:
df['diagAct'] = df.apply(lambda x: [tagger.tag(xseq) for xseq in createDiagActFeatures(str(x.text))][0][0], axis=1)
mostCommonDiagAct = collections.Counter(df.diagAct.values.tolist()).most_common(8)
mostCommonDiagAct = [i[0] for i in mostCommonDiagAct]
def helper(a):
    if a not in mostCommonDiagAct:
        return 'othr'
    else: return a
df['diagAct'] = df.apply(lambda x: helper(x.diagAct),axis=1)
df['diagAct'].value_counts()

sd                 2798
othr               1714
qy                 1563
qw                 1374
sv                 1014
qy^d                500
^q                  423
ad                  382
fo_o_fw_"_by_bc     345
Name: diagAct, dtype: int64

In [11]:

# One hot encoding
df = pd.get_dummies(df, columns=["diagAct"], prefix=["diagAct"])
# label encoding
# encoder = LabelEncoder()
# diagAct = encoder.fit_transform(df['diagAct'])
# df['diagActEnc'] = diagAct
df.shape

(10113, 12)

In [12]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import words
stopwords = stopwords.words('english')
porter = PorterStemmer()
def normalize_text(s):
    # just in case
    s = str(s)
    s = s.lower()
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W\s',' ',s)
    
    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    return s


def token_normalize_text(text):
    #print "original:", text
    # Remove Emails
#     text = re.sub(r'\S*@\S*\s?', '', text)
#     # Remove website links
#     text = re.sub(r'http[s]?://\S+', '', text)
    # Remove distracting single quotes
    text = re.sub(r"\'", "", text)
    # Remove distracting double quotes
    text = re.sub(r'\"', "", text)
    # Remove new line characters
    text = re.sub(r'\s+', ' ', text)
    # word normalisation
    text = re.sub(r"(\w)([.,;:!?'/\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'/\"“\(])(\w)", r"\1 \2", text)
    # normalisation
    text = re.sub(r"(\S)\1\1+",r"\1\1\1", text)
    #tokenising
    
    tokens = list(flatten([re.split(r"\s+",t) for t in re.split('(\d+)',text)]))
    tokens = [re.sub(r'[^A-Za-z]+','',t) for t in tokens]
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in ' ' and len(t) > 2]
    tokens = [w for w in tokens if w not in stopwords ]
    tokens = [str(porter.stem(w)) for w in tokens]
    return ' '.join(tokens)


df['text_norm'] = df.apply(lambda x: normalize_text(str(x.text)),axis = 1)
df['token_text_norm'] = df.apply(lambda x: token_normalize_text(str(x.text)),axis = 1)

In [13]:
def GetFMeasure(text):
    tagged = GetPOSTag(text)

    freq = {}
    freq['noun'] = 0
    freq['adj'] = 0
    freq['prep'] = 0
    freq['art'] = 0
    freq['pron'] = 0
    freq['verb'] = 0
    freq['adv'] = 0
    freq['int'] = 0

    count = 0
    for i in range(len(tagged)):
        pos = tagged[i]
        if pos in ['NN', 'NNS', 'NNP', 'NNPS']:
            freq['noun'] += 1
        elif pos in ['JJ', 'JJR', 'JJS']:
            freq['adj'] += 1
        elif pos in ['IN']:
            freq['prep'] += 1
        elif pos in ['DET', 'DT', 'PDT', 'WDT']:
            freq['art'] += 1
        elif pos in ['PRP', 'PRP$', 'WP', 'WP$']:
            freq['pron'] += 1
        elif pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            freq['verb'] += 1
        elif pos in ['RB', 'RBR', 'RBS', 'WRB']:
            freq['adv'] += 1
        elif pos in ['UH']:
            freq['int'] += 1

        if pos not in ['$', "'", '(', ')', ',', '-', '.', ':', 'SYM', "''", '``']:
            count += 1

    for key in freq:
        freq[key] = (freq[key] / count) * 100

    fmeasure = 0.5 * ( (freq['noun'] + freq['adj'] + freq['prep'] + freq['art']) - (freq['pron'] + freq['verb'] + freq['adv'] + freq['int']) + 100 )

    return fmeasure

In [14]:
df['F-Measure'] = df.apply(lambda x: GetFMeasure(x.text_norm), axis=1)

In [15]:
tokens=[]
for i in df.token_text_norm.values.tolist():
    tokens.append(i.split())
tokens = [i for i in flatten(tokens)]

counterDict = collections.Counter(tokens)
meantokenperdiag=np.mean(df.apply(lambda x: len(str(x.token_text_norm).split()),axis=1))
total_no_diag = df.shape[0]
total_no_of_clean_words = sum(counterDict.values())
print("Total number of tokens: ", total_no_of_clean_words)
print("Total number of unique tokens: ", len(counterDict))
print("Total Number of dialouge: ", total_no_diag)
print("Mean token length: ", meantokenperdiag)

print("Most common tokens",counterDict.most_common(10))
featureDict = {} # A global dictionary of features

print("Building feature dict")
featureDict ={}
for k,v in counterDict.items():
	featureDict[k] = (float(v)/total_no_of_clean_words)*math.log(float(total_no_diag) / v)

df_old = df.copy()
x_y = []
for index, rows in df.iterrows():
    token = str(rows['token_text_norm']).split()
    f =[]
    for vals in range(int(meantokenperdiag)+2):
        
        if len(token)>=int(meantokenperdiag)+2:
            #do not pad
            f.append(featureDict[token[vals]])
        else:
            #pad with 0
            if vals < len(token):
                f.append(featureDict[token[vals]])
            else:
                f.append(0.0)
    #f.append(diag)
    x_y.append(f)

df1 = pd.DataFrame(x_y,columns=['tf1','tf2','tf3','tf4','tf5','tf6'])

Total number of tokens:  49227
Total number of unique tokens:  4123
Total Number of dialouge:  10113
Mean token length:  4.867695045980422
Most common tokens [('dont', 890), ('yeah', 793), ('know', 787), ('get', 734), ('well', 675), ('got', 568), ('want', 560), ('like', 555), ('your', 546), ('look', 497)]
Building feature dict


In [16]:
df_new = pd.concat([df,df1],axis = 1)

In [29]:
df_new.to_csv("features.csv",index=False)
df_new

,text,character,gender,diagAct_^q,diagAct_ad,"diagAct_fo_o_fw_""_by_bc",diagAct_othr,diagAct_qw,diagAct_qy,diagAct_qy^d,...,diagAct_sv,text_norm,token_text_norm,F-Measure,tf1,tf2,tf3,tf4,tf5,tf6
0,Someone had fun.,SEAN,male,0,0,0,0,0,0,0,...,0,someone had fun.,someon fun,50.000000,0.008708,0.003150,0.000000,0.000000,0.000000,0.000000
1,"It's no problem, honestly. Go on, go and open ...",SHIRLEY,female,0,1,0,0,0,0,0,...,0,it's no problem honestly go on go and open the...,problem honestli open launderett leav,55.882353,0.005826,0.001406,0.004029,0.000906,0.011087,0.000000
2,Last night was better than ever. What's all th...,MAX,male,0,0,0,0,0,1,0,...,0,last night was better than ever what's all thi...,last night better ever what anyth interest,57.692308,0.010809,0.014854,0.010809,0.007072,0.015701,0.011971
3,Have you checked the answerphone? Any calls?,IAN,male,0,0,0,0,0,1,0,...,0,have you checked the answerphone any calls?,check answerphon call,62.500000,0.004123,0.000187,0.010387,0.000000,0.000000,0.000000
4,Oscar's asleep.,MAX,male,0,0,0,1,0,0,0,...,0,oscar's asleep.,oscar asleep,50.000000,0.004496,0.001035,0.000000,0.000000,0.000000,0.000000
5,Dead Arm? What's Dead Arm?,MINTY,male,0,0,0,1,0,0,0,...,0,dead arm what's dead arm?,dead arm what dead arm,50.000000,0.002844,0.002096,0.015701,0.002844,0.002096,0.000000
6,Rox...,RONNIE,female,0,0,0,0,1,0,0,...,0,rox...,rox,50.000000,0.002206,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Oh no, I wouldn't do that. Here's the menu. Sh...",IAN,male,0,0,0,0,0,0,0,...,0,oh no i wouldn't do that here's the menu shoul...,wouldnt here menu perfectli within capabl,55.555556,0.008783,0.001161,0.001035,0.001035,0.000346,0.000346
8,I think I've got someone in line already.,RONNIE,female,0,0,0,0,0,0,0,...,0,i think i've got someone in line already.,think ive got someon line alreadi,50.000000,0.027802,0.024393,0.033224,0.008708,0.002423,0.005480
9,Like drinking yourself to death?,SEAN,male,0,0,0,0,0,1,0,...,0,like drinking yourself to death?,like drink death,50.000000,0.032725,0.010316,0.001035,0.000000,0.000000,0.000000


In [35]:
df_new = pd.read_csv("data/features_gender.csv")

In [37]:
encoder = LabelEncoder()
y = encoder.fit_transform(df_new['gender'])
# x = df_new.drop(['text', 'character','text_norm','token_text_norm','gender', "tf1","tf2","tf3","tf4","tf5","tf6"],axis=1)
x = df_new.drop(['gender'],axis =1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8090, 53)
(8090,)
(2023, 53)
(2023,)


In [ ]:
x_train

In [38]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

print(nb.score(x_test, y_test))

# clf = svm.SVC(kernel='linear', C=1)
# scores = cross_val_score(clf, x_train, y_train, cv=5)
# scores
# clf = svm.SVC(kernel='linear', C=1)
# scores = cross_val_score(clf, x_train, y_train, cv=5)
# scores


ValueError: could not convert string to float: 'You should have told me.'

In [20]:
# pull the data into vectors
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 3, stop_words='english')
x = vectorizer.fit_transform(df_new['token_text_norm'])
df1 = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names())
# df.drop('text', axis=1, inplace=True)
res = pd.concat([df_new.drop(['text', 'character','text_norm','token_text_norm','gender'],axis=1), df1], axis=1)



encoder = LabelEncoder()
y = encoder.fit_transform(df['gender'])

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(res, y, test_size=0.2)

# take a look at the shape of each of these
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8090, 1596)
(8090,)
(2023, 1596)
(2023,)


In [21]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

print(nb.score(x_test, y_test))

0.5823035096391498


In [22]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, x_train, y_train, cv=5)
scores                                              


array([0.53798641, 0.5512979 , 0.54388133, 0.54202719, 0.55782313])

In [23]:
# pull the data into vectors
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(df['text_norm'])

encoder = LabelEncoder()
y = encoder.fit_transform(df['character'])

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# take a look at the shape of each of these
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8090, 5548)
(8090,)
(2023, 5548)
(2023,)


In [24]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

print(nb.score(x_test, y_test))

0.17301038062283736


In [25]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, x_train, y_train, cv=5)
scores      

array([0.20492308, 0.19173859, 0.20259419, 0.20619195, 0.21006837])